# DDPLKO Moduł 12 - praca domowa - Wdrażanie modelu

To dwunasta praca domowa w Programie szkoleniowym Klasyfikacja obrazu od Deep Drive PL

Twoim zadaniem w tym module będzie:
- [ ] Wdrożenie jednego ze swoich modeli jako aplikację demo: modelu do klasyfikacji rysunków z QuickDraw, modelu wytrenowanego z wykorzystaniem transfer learningu, modelu do klasyfikacji binarnej
- [ ] Wykorzystaj np. FastAPI, Streamlit lub Gradio App
- [ ] Udostępnij screenshot wdrożonej aplikacji na Discordzie `#klasyfikacja-wyniki`

Extra - dodatkowo możesz:
- Skorzystać z technik takich jak Torch Serve, TensorFlow Serving czy Nvidia Triton Inference server

Jako rozwiązanie prześlij kod źródłowy z którego korzystasz - notebook/skrypt

# Model

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.nn import functional as F
from torch.utils.data import TensorDataset, DataLoader
from sklearn.metrics import accuracy_score

import pytorch_lightning as pl

In [2]:
from kornia.augmentation import RandomHorizontalFlip, RandomVerticalFlip, RandomRotation

In [3]:
class DataAugmentation(nn.Module):
    def __init__(self, apply_color_jitter: bool = False) -> None:
        super().__init__()
        self._apply_color_jitter = apply_color_jitter

        self.transforms = nn.Sequential(
            RandomHorizontalFlip(p=0.5),
            RandomVerticalFlip(p=0.5),
            RandomRotation(p=0.15, degrees=[-180, 180])
        )

    @torch.no_grad()
    def forward(self, x):
        x_out = self.transforms(x)
        return x_out

In [70]:
class QuickDrawClassifier(pl.LightningModule):
    def __init__(self):
        super().__init__()
        self.model = nn.Sequential(
            nn.Conv2d(1, 32, 3),
            nn.ReLU(),
            nn.BatchNorm2d(32),
            nn.Dropout(0.3),

            nn.AvgPool2d(2),
            nn.Conv2d(32, 128, 3),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.BatchNorm2d(128),

            nn.Conv2d(128, 64, 3),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.BatchNorm2d(64),
        
            nn.Flatten(),
            nn.Linear(5184, 10))

        #self.transforms = DataAugmentation()
  
    def forward(self, x):
        y_hat = self.model(x)
        return y_hat
    
    def configure_optimizers(self):
        optimizer = optim.Adam(self.model.parameters(), lr=1e-3)
        return optimizer
    
    def training_step(self, train_batch, batch_idx):
        x, y = train_batch
        x# = self.transforms(x)
        y_hat = self.model(x)
        
        loss = nn.CrossEntropyLoss()(y_hat, y)
        pred = y_hat.argmax(dim=1, keepdim=True)
        acc = accuracy_score(y.detach().cpu().numpy(), pred.detach().cpu().numpy())

        return {'loss': loss, 'accuracy': acc}

    def validation_step(self, val_batch, batch_idx):
        x, y = val_batch
        y_hat = self.model(x)
        loss = nn.CrossEntropyLoss()(y_hat, y)
        pred = y_hat.argmax(dim=1, keepdim=True)
        acc = accuracy_score(y.detach().cpu().numpy(), pred.detach().cpu().numpy())

        return {'loss': loss, 'accuracy': acc}

    def training_epoch_end(self, outputs):
        mean_loss = torch.stack([x['loss'] for x in outputs]).mean()
        mean_acc = np.stack([x['accuracy'] for x in outputs]).mean()
        self.log('loss_train', mean_loss.detach().cpu().item())
        self.log('acc_train', mean_acc.item())

    def validation_epoch_end(self, outputs):
        mean_loss = torch.stack([x['loss'] for x in outputs]).mean()
        mean_acc = np.stack([x['accuracy'] for x in outputs]).mean()
        self.log('loss_val', mean_loss.detach().cpu().item())
        self.log('acc_val', mean_acc.item())

In [5]:
activation = {}


def get_activation(name):

    def hook(model, input, output):
        activation[name] = output.detach()

    return hook

In [72]:
import urllib.request
import numpy as np
import matplotlib.pyplot as plt
import os

class_names = [
    'spider', 'penguin', 'shark', 'sheep', 'rhinoceros', 'hedgehog', 'horse',
    'fish', 'flamingo', 'frog'
]
for name in class_names:
    url = 'https://storage.googleapis.com/quickdraw_dataset/full/numpy_bitmap/%s.npy' % name
    file_name = url.split('/')[-1].split('?')[0]

    url = url.replace(' ', '%20')

    if not os.path.isfile(file_name):
        print(url, '==>', file_name)
        urllib.request.urlretrieve(url, file_name)

data = []
for name in class_names:
    file_name = name + '.npy'
    data.append(np.load(file_name, fix_imports=True, allow_pickle=True))
    print('%-15s' % name, type(data[-1]))

for i in range(len(data)):
    data[i] = data[i][::100]

data = [np.reshape(name, (-1, 28, 28)) for name in data]

X = np.concatenate(data).reshape(-1, 1, 28, 28)
X = X = X.astype('float32')

y = np.concatenate([np.array([i] * len(_)) for i, _ in enumerate(data)],
                   axis=0)
y = y = y.astype('int64')
X = X / 255


spider          <class 'numpy.ndarray'>
penguin         <class 'numpy.ndarray'>
shark           <class 'numpy.ndarray'>
sheep           <class 'numpy.ndarray'>
rhinoceros      <class 'numpy.ndarray'>
hedgehog        <class 'numpy.ndarray'>
horse           <class 'numpy.ndarray'>
fish            <class 'numpy.ndarray'>
flamingo        <class 'numpy.ndarray'>
frog            <class 'numpy.ndarray'>


In [64]:
from PIL import Image

im = (X[0][0] * 255).astype(np.uint8)
im = Image.fromarray(im)
im.save("spider.jpg")

In [73]:
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")

batch_size = 2048
dataset = TensorDataset(torch.Tensor(X), torch.Tensor(y).type(torch.long))
del X, y, data

train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size

train_set, val_set = torch.utils.data.random_split(dataset,
                                                   [train_size, test_size])

train_loader = DataLoader(train_set, batch_size=batch_size, num_workers=20)
val_loader = DataLoader(val_set, batch_size=batch_size, num_workers=20)

In [ ]:
model = QuickDrawClassifier()

trainer = pl.Trainer(gpus=1,
                     precision=16,
                     max_epochs=50,
                     num_sanity_val_steps=0)
trainer.fit(model, train_loader, val_loader)

In [26]:
%reload_ext tensorboard
%tensorboard --logdir=lightning_logs/

In [75]:
torch.save(model, 'model_small.pt')

# Gradio


In [7]:
import gradio as qr
from captum.attr import Occlusion, GuidedGradCam
from captum.attr import visualization as viz

test_image = None
model = torch.load('model_90_proc.pt')

In [24]:
import gradio as gr
from skimage.transform import resize
import PIL

strides = 1
sliding_window_shape = (1, 4, 4)
baselines = 0


def sketch_recognition(img):
    x = torch.tensor(img, dtype=torch.float32).unsqueeze(0).unsqueeze(0) / 255.
    with torch.no_grad():
        out = model(x)
    probabilities = torch.nn.functional.softmax(out[0], dim=0)
    values, indices = torch.topk(probabilities, 5)
    confidences = {class_names[i]: v.item() for i, v in zip(indices, values)}
    occlusion = Occlusion(model)
    attributions = occlusion.attribute(
        x,
        strides=strides,
        target=indices[0],
        sliding_window_shapes=sliding_window_shape,
        baselines=baselines,
        show_progress=False)
    attributions = attributions.squeeze().cpu().detach().numpy()
    attributions = np.stack((attributions, ) * 3, axis=2)
    attributions -= np.min(attributions)
    attributions /= np.max(attributions)
    attributions += np.stack((img, ) * 3, axis=2) / 255
    attributions /= np.max(attributions)

    return confidences, attributions


gr.Interface(fn=sketch_recognition,
             inputs="sketchpad",
             outputs=["label", "image"]).launch(debug=True)


Running on local URL:  http://127.0.0.1:7860/

To create a public link, set `share=True` in `launch()`.


Keyboard interruption in main thread... closing server.


(<gradio.routes.App at 0x7fc6fc455dc0>, 'http://127.0.0.1:7860/', None)

In [28]:
# Torch Serve
import os
import pathlib

In [119]:
pathlib.Path("model_store").mkdir(parents=True, exist_ok=True)

model = torch.load("model_small.pt")
script = model.to_torchscript()
img = torch.zeros(1,1,28,28)

traced_model = torch.jit.trace(script, img)
traced_model.save('model_store/model_small.pt')

In [120]:
!tree model_store

model_store
└── model_small.pt

0 directories, 1 file


In [121]:
! torch-model-archiver --export-path model_store --model-name model_small \
                       --version 1.0 --serialized-file model_store/model_small.pt \
                       --extra-files index_to_name.json --handler handler.py --force

In [34]:
!torchserve --start --ncs --model-store model_store --models quick_draw_classifier.mar

In [123]:
! curl http://127.0.0.1:8080/predictions/model_small -T spider.jpg

{
  "spider": 0.5955545902252197,
  "hedgehog": 0.28811168670654297,
  "horse": 0.11632220447063446,
  "penguin": 8.179895303328522e-06,
  "sheep": 2.9395096134976484e-06
}

# Wyślij rozwiązanie
Możesz skorzystać z jednego z poniższych sposobów:
**mailem na specjalny adres** ze strony pracy domowej w panelu programu prześlij jedno z poniższych:
- notebooka (jeżeli plik ma mniej niż np. 10MB)
- notebooka w zipie
- link do Colaba (udostępniony)
- link do pliku przez GDrive/Dropboxa/WeTransfer/...
- pdfa (poprzez download as pdf)
- jako plik w repozytorium na np. GitHubie, by budować swoje portfolio (wtedy uważaj na wielkość pliku, najlepiej kilka MB, Max 25MB)

Najlepiej, by w notebooku było widać wyniki uruchomienia komórek, chyba, że przez nie plik będzie mieć 100+MB wtedy najlepiej Colab lub jakieś przemyślenie co poszło nie tak (zbyt dużo dużych zdjęć wyświetlonych w komórkach).

## Co otrzymasz?
Informację zwrotną z ewentualnymi sugestiami, komentarzami.